In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kiolpujy/class4.csv
/kaggle/input/kiolpujy/class3.csv
/kaggle/input/kiolpujy/class1.csv
/kaggle/input/kiolpujy/class6.csv
/kaggle/input/kiolpujy/class0.csv
/kaggle/input/kiolpujy/class5.csv
/kaggle/input/kiolpujy/class2.csv


In [2]:
pip install wtfml


Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import torch
import albumentations

import numpy as np
import pandas as pd

import torch.nn as nn
from sklearn import metrics
from sklearn import model_selection
from torch.nn import functional as F

from wtfml.utils import EarlyStopping
from wtfml.engine import Engine

In [4]:
import torch
import torch.nn as nn
import torchvision.models as models
class ResNetForMultiClass(nn.Module):
    def __init__(self, num_classes=2, pretrained='imagenet'):
        super(ResNetForMultiClass, self).__init__()
        
        # Load the pre-trained ResNet model
        self.base_model = models.resnet152(pretrained=(pretrained == 'imagenet'))
        
        # Replace the final fully connected layer of ResNet
        self.base_model.fc = nn.Linear(in_features=self.base_model.fc.in_features, out_features=512)
        
        # Add the output layer for multi-class classification
        self.out = nn.Linear(512, num_classes)
        
        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, image, targets=None):
        # Pass the image through the base ResNet model
        x = self.base_model(image)
        
        # Apply ReLU and Dropout
        x = torch.relu(x)
        if targets is not None:
            x = self.dropout(x)
        
        # Output layer
        out = self.out(x)
        
        # If targets are provided (training mode), compute the loss
        if targets is not None:
            loss = FocalLoss()(out, targets)
            return out, loss
        
        # If no targets are provided (inference mode), return only the output
        return out

In [5]:
import torch
from torch.utils.data import Dataset
import numpy as np
import cv2
import albumentations as A

class CustomImageDataset(Dataset):
    def __init__(self, pixel_arrays, targets, resize=None, augmentations=None):
        self.pixel_arrays = pixel_arrays
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations

    def __len__(self):
        return len(self.pixel_arrays)

    def __getitem__(self, idx):
        # Convert the 1D array to a 2D grayscale image
        image = np.array(self.pixel_arrays[idx], dtype=np.float32).reshape(48, 48)
        
        # Resize image if needed
        if self.resize:
            image = cv2.resize(image, self.resize)

        # Expand dimensions to (48, 48, 1) and then convert to (new_size, new_size, 3) for RGB
        image = np.expand_dims(image, axis=-1)
        image = np.repeat(image, 3, axis=-1)  # Convert to RGB

        # Apply augmentations if specified
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']

        # Convert image to PyTorch tensor and permute to (3, new_size, new_size) format
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)  # For RGB, shape becomes (3, new_size, new_size)
        target = torch.tensor(self.targets[idx], dtype=torch.long)

        return image, target
import torch
import numpy as np
from sklearn import metrics
from tqdm import tqdm

class Engine:
    @staticmethod
    def train(data_loader, model, optimizer, device, scheduler=None, accumulation_steps=1, fp16=False):
        model.train()
        losses = AverageMeter()
        scaler = torch.cuda.amp.GradScaler() if fp16 else None
        
        if accumulation_steps > 1:
            optimizer.zero_grad()
        
        tk0 = tqdm(data_loader, total=len(data_loader))
        for batch_idx, (images, targets) in enumerate(tk0):
            images = images.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                if fp16:
                    with torch.cuda.amp.autocast():
                        outputs = model(images)
                        loss = FocalLoss()(ouputs,targets)
                    scaler.scale(loss).backward()
                else:
                    outputs = model(images)
                    loss = FocalLoss()(outputs, targets)
                    loss.backward()

                if (batch_idx + 1) % accumulation_steps == 0:
                    scaler.step(optimizer) if fp16 else optimizer.step()
                    if scheduler:
                        scheduler.step()
                    optimizer.zero_grad()

            losses.update(loss.item(), data_loader.batch_size)
            tk0.set_postfix(loss=losses.avg)
        
        return losses.avg

    @staticmethod
    def evaluate(data_loader, model, device, use_tpu=False):
        losses = AverageMeter()
        final_predictions = []
        model.eval()
        with torch.no_grad():
            tk0 = tqdm(data_loader, total=len(data_loader), disable=use_tpu)
            for b_idx, data in enumerate(tk0):
                images, targets = data  # Adjust if your data format is different

                # Move tensors to device
                images = images.to(device)
                targets = targets.to(device)

                # Forward pass
                predictions, loss = model(images, targets)
                predictions = predictions.cpu()
                losses.update(loss.item(), images.size(0))
                final_predictions.append(predictions)
                tk0.set_postfix(loss=losses.avg)

        # Concatenate all predictions and convert to NumPy array
        final_predictions = torch.cat(final_predictions).numpy()
        return final_predictions, losses.avg
    def predict(data_loader, model, device, use_tpu=False):
        model.eval()
        final_predictions = []
        with torch.no_grad():
            tk0 = tqdm(data_loader, total=len(data_loader), disable=use_tpu)
            for b_idx, data in enumerate(tk0):
                inputs, _ = data  # Unpack data
                inputs = inputs.to(device)
                predictions = model(inputs) 
            # Assume model returns only predictions
                final_predictions.append(predictions.cpu())
                tk0.set_postfix()
        return torch.cat(final_predictions).numpy()
        # Concatenate all predictions and convert to numpy array
        return torch.cat(final_predictions).numpy()
class AverageMeter():
        def __init__(self):
            self.reset()

        def reset(self):
            self.val = 0
            self.avg = 0
            self.sum = 0
            self.count = 0

        def update(self, val, n=1):
            self.val = val
            self.sum += val * n
            self.count += n
            self.avg = self.sum / self.count

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = 1e-12  # prevent training from NaN-loss errors

    def forward(self, logits, target):
        """
        logits: [batch_size, num_classes]
        target: [batch_size] (true class indices, not one-hot)
        """
        # Compute softmax over logits to get the class probabilities
        probs = F.softmax(logits, dim=1)
        
        # Gather the probabilities of the true class (index-based selection)
        probs_target_class = probs.gather(1, target.unsqueeze(1)).squeeze(1) + self.epsilon
        
        # Compute the log of the selected probabilities
        log_pt = torch.log(probs_target_class)
        
        # Calculate the focal loss (focuses more on hard examples)
        focal_loss = -1 * self.alpha * (1 - probs_target_class) ** self.gamma * log_pt
        
        return torch.mean(focal_loss)

In [7]:
import albumentations as A
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
def train(fold):
    df = pd.read_csv(f"/kaggle/input/kiolpujy/class{fold}.csv")
    df_train, df_valid = train_test_split(df, test_size=0.2, random_state=42, stratify=df[f'class_{fold}'])
    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)
    device = "cuda"
    epochs = 50
    train_bs = 32
    valid_bs = 16
    train_pixels = df_train['pixels'].apply(lambda x: np.fromstring(x, sep=' ', dtype=np.float32))
    train_targets = df_train[f'class_{fold}'].values

    valid_pixels = df_valid['pixels'].apply(lambda x: np.fromstring(x, sep=' ', dtype=np.float32))
    valid_targets = df_valid[f'class_{fold}'].values

    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    train_aug = A.Compose(
    [
        A.Resize(height=299, width=299),  # Resize images to 224x224
        A.Normalize(mean=mean, std=std, max_pixel_value=255.0, always_apply=True),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15),
        A.HorizontalFlip(p=0.5)
    ]
    )

    valid_aug = A.Compose(
        [
        A.Resize(height=299, width=299),  # Resize images to 224x224
        A.Normalize(mean=mean, std=std, max_pixel_value=255.0, always_apply=True)
        ]
    )
    train_dataset = CustomImageDataset(
        pixel_arrays=train_pixels,
        targets=train_targets,
        resize=(299, 299),  # New size
        augmentations=train_aug,
    )

    train_loader = DataLoader(
        train_dataset, batch_size=32, shuffle=True, num_workers=4
    )

    valid_dataset = CustomImageDataset(
        pixel_arrays=valid_pixels,
        targets=valid_targets,
        resize=(299, 299),  # New size
        augmentations=valid_aug,
    )

    valid_loader = DataLoader(
        valid_dataset, batch_size=32, shuffle=False, num_workers=4
    )
    model = ResNetForMultiClass(num_classes= 7,pretrained='imagenet')
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        patience=3,
        threshold=0.001,
        mode="max"
    )

    es = EarlyStopping(patience=5, mode="max")
    for epoch in range(epochs):
        train_loss = Engine.train(train_loader, model,optimizer,
        device,
        scheduler=None,
        accumulation_steps=1,
        fp16=False)
        predictions, valid_loss = Engine.evaluate(
            valid_loader, model, device=device
        )
        final_predictions = np.argmax(np.vstack(predictions), axis=1)
        valid_targets = np.array(valid_targets)  # Ensure targets are in the right format
    
    # Calculate accuracy
        accuracy = metrics.accuracy_score(valid_targets, final_predictions)
        print(f"Epoch = {epoch}, Accuracy = {accuracy:.4f}")
    
    # Update the learning rate scheduler
        scheduler.step(accuracy)
    
    # Early Stopping
        es(accuracy, model, model_path=f"model_fold_RES152{fold}.bin")
        if es.early_stop:
            print("Early stopping")
            break
    oof_data = {
        'id': df_valid.index,  # Use the index to map back to the original data
        'true_emotion': valid_targets,
        'pred_emotion': final_predictions,
        'fold': fold
    }
    
    return pd.DataFrame(oof_data)

In [8]:
X = np.empty((0, 4))  # Initialize an empty array with 7 columns to match df.values

for i in range(0, 5):
    df = train(i)  # Assuming train(1) returns a DataFrame with 7 columns
    df.to_csv(f"oof1EFOCAL-{i}.csv", index=False, header=False)
    if len(df.columns)!=4:
        df = df.drop('Unnamed: 0',axis = 1)    
    X = np.concatenate((X, df.values))  # Concatenate new data to X

# Convert the final result into a DataFrame and save to CSV
df = pd.DataFrame(X)
df.to_csv('RES152B', index=False)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:01<00:00, 157MB/s]
100%|██████████| 32/32 [00:13<00:00,  2.45it/s, loss=0.112]


Epoch = 0, Accuracy = 0.8350
Validation score improved (-inf --> 0.835). Saving model!


100%|██████████| 32/32 [00:13<00:00,  2.44it/s, loss=0.0897]


Epoch = 1, Accuracy = 0.8610
Validation score improved (0.835 --> 0.861). Saving model!


100%|██████████| 32/32 [00:13<00:00,  2.44it/s, loss=0.0976]


...

Epoch = 8, Accuracy = 0.8370
EarlyStopping counter: 3 out of 5


100%|██████████| 32/32 [00:13<00:00,  2.45it/s, loss=0.157]


Epoch = 9, Accuracy = 0.7610
EarlyStopping counter: 4 out of 5


100%|██████████| 32/32 [00:13<00:00,  2.45it/s, loss=0.13]


Epoch = 10, Accuracy = 0.8340
EarlyStopping counter: 5 out of 5
Early stopping
